In [1]:
import pandas as pd
import re
import openai

In [65]:
# Read the TSV file into a DataFrame
# df = pd.read_csv('../public/teacherstudentchat00256.tsv', sep='\t')
df = pd.read_csv('../public/teacherstudentchat00006.tsv', sep='\t')

# Extract the ’edited‘ column
edited = df['edited']
print(edited)

role = df['role']
print(role)

0                            Hi <STUDENT>, is that you?!
1                                           Hi <TEACHER>
2               I can see more people here, is it right?
3                              Hi there, how's it going?
4                              No it's only us I think..
                             ...                        
133    Yes OK thanks <STUDENT> - I'll say goodbye - I...
134    I got a bit crazy about the sun when I lived t...
135    I can imagine if you're used to that kind  of ...
136                                              Bye bye
137                                  Thank you very much
Name: edited, Length: 138, dtype: object
0      teacher
1      student
2      student
3      teacher
4      teacher
        ...   
133    teacher
134    student
135    teacher
136    student
137    student
Name: role, Length: 138, dtype: object


In [66]:
# Combine the columns into the desired format
combined_text = ""
for r, e in zip(role, edited):
    combined_text += f"{r}: {e}\n"

print(combined_text)

teacher: Hi <STUDENT>, is that you?!
student: Hi <TEACHER>
student: I can see more people here, is it right?
teacher: Hi there, how's it going?
teacher: No it's only us I think..
student: I'm very well, thank you. How about you?
teacher: Yeah fine thanks...not so busy at the moment to be honest. I finished the teacher training course stuff
student: That's good!!
teacher: Yes, I've got to buy a car, which is fine but a bit stressful you know
teacher: I'm trying to decide between the crap tight-fisted option or a newer more xpensive car
student: Ugh, I can imagine. I think it takes time, looking for the one you like and trying different ones... (do you say 'trying' for cars?)
teacher: You say 'try out' for a car - a good phrasal verb!
student: Thank you!
teacher: But you can just say try of course
teacher: It just makes it slightly less formal
teacher: to use the phrasal verb i mean
student: I see
teacher: But very very common in speech of course
teacher: So I tried a few - 'had a test d

In [67]:
# Function to highlight background of text within <>
def highlight_background(text):
    return re.sub(r'(<[^>]+>)', r'\033[42;30m\1\033[0m', text)

# Combine the columns into the desired format with highlighted background text
combined_text = ""
for r, e in zip(role, edited):
    highlighted_edited = highlight_background(e)
    combined_text += f"{r}: {highlighted_edited}\n"

print(combined_text)

teacher: Hi <STUDENT>, is that you?!
student: Hi <TEACHER>
student: I can see more people here, is it right?
teacher: Hi there, how's it going?
teacher: No it's only us I think..
student: I'm very well, thank you. How about you?
teacher: Yeah fine thanks...not so busy at the moment to be honest. I finished the teacher training course stuff
student: That's good!!
teacher: Yes, I've got to buy a car, which is fine but a bit stressful you know
teacher: I'm trying to decide between the crap tight-fisted option or a newer more xpensive car
student: Ugh, I can imagine. I think it takes time, looking for the one you like and trying different ones... (do you say 'trying' for cars?)
teacher: You say 'try out' for a car - a good phrasal verb!
student: Thank you!
teacher: But you can just say try of course
teacher: It just makes it slightly less formal
teacher: to use the phrasal verb i mean
student: I see
teacher: But very very common in speech of course
teacher: So I tried a few - 'had a test d

In [68]:
# z = re.findall('.*(<.*>).*', "student: How did <TEACHER'S CHILD> take <TEACHER> it ?")
# # print(z)
# print(z.group(0))

1. Replacement using GPT
2. run Presidio (performance might be perfect)
3. Adversarial attack (randomly add PII, then Presidio)
4. John -> DohN (then Presidio，performance might be lower)

In [69]:
# !%pip install --upgrade openai

zsh:fg:1: no job control in this shell.


In [70]:
# Set your OpenAI API key
openai.api_key = 'your-api-key'

# Function to generate synthetic data
def generate_synthetic_data(category):
    # Define specific prompts for each category
    prompts = {
        "teacher": "Generate a realistic teacher name. Just give me one name with no additional output.",
        "student": "Generate a realistic student name. Just give me one name with no additional output.",
        "dob": "Generate a realistic date of birth. Just give me one date of birth with no additional output.",
        "address": "Generate a realistic address. Just give me one address with no additional output.",
        "telephone": "Generate a realistic telephone number. Just give me one telephone number with no additional output.",
        "email": "Generate a realistic email address. Just give me one email address with no additional output.",
        "teacher's child": "Generate a realistic teacher's child name. Just give me one name with no additional output.",
        "student's child": "Generate a realistic student's child name. Just give me one name with no additional output.",
    }
    
    # Select the appropriate prompt based on the category
    prompt = prompts.get(category.lower(), f"Generate a realistic {category.lower()}. Just give me one result with no additional output.")
    
    # Generate synthetic data using the OpenAI API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message['content'].strip()

# Function to replace placeholders
def replace_placeholders(text):
    pattern = re.compile(r'<([^>]+)>')
    placeholders = pattern.findall(text)
    synthetic_data = {}
    
    for placeholder in placeholders:
        if placeholder not in synthetic_data:
            # Generate and store synthetic data for each unique placeholder category
            synthetic_data[placeholder] = generate_synthetic_data(placeholder)
    
    # Replace all instances of each placeholder with the generated synthetic data
    for placeholder, synthetic in synthetic_data.items():
        text = text.replace(f"<{placeholder}>", synthetic)
    
    return text

# Example text with placeholders
# example_text = """Hi <TEACHER>, please contact <STUDENT> at <EMAIL>. My date of birth is <DOB> and I live at <ADDRESS>. 
# You can call me at <TELEPHONE>. <STUDENT> has an appointment with <TEACHER> tomorrow."""

replaced_text = replace_placeholders(combined_text)
print(replaced_text)


teacher: Hi Emily Johnson, is that you?!
student: Hi Ms. Jennifer Reynolds
student: I can see more people here, is it right?
teacher: Hi there, how's it going?
teacher: No it's only us I think..
student: I'm very well, thank you. How about you?
teacher: Yeah fine thanks...not so busy at the moment to be honest. I finished the teacher training course stuff
student: That's good!!
teacher: Yes, I've got to buy a car, which is fine but a bit stressful you know
teacher: I'm trying to decide between the crap tight-fisted option or a newer more xpensive car
student: Ugh, I can imagine. I think it takes time, looking for the one you like and trying different ones... (do you say 'trying' for cars?)
teacher: You say 'try out' for a car - a good phrasal verb!
student: Thank you!
teacher: But you can just say try of course
teacher: It just makes it slightly less formal
teacher: to use the phrasal verb i mean
student: I see
teacher: But very very common in speech of course
teacher: So I tried a few